In [15]:

import torch

print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())


2.5.1+cu121
CUDA available: True
CUDA device count: 1


In [30]:
import pandas as pd
LONG_METHOD_dataset = pd.read_csv("..\..\..\dataset-source\embedding-dataset\software-metrics\LongMethod_code_metrics_values.csv")
X_LONG_METHOD_dataset,y_LONG_METHOD_dataset = LONG_METHOD_dataset.drop(['sample_id','label'],axis=1),LONG_METHOD_dataset['label']

print(X_LONG_METHOD_dataset.shape)

print(y_LONG_METHOD_dataset.shape)


(2408, 26)
(2408,)


In [32]:
input_features = X_LONG_METHOD_dataset.shape[1]
input_features

26

In [33]:
### creating simple ann model

import torch 
import torch.nn as nn
import torch.nn.functional as F

class ANN_Model(nn.Module):
    def __init__(self, input_features = input_features, hidden1 =20, hidden2 =20, out_features = 1):
        super().__init__()
        
        self.f_connected1 = nn.Linear(input_features,hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self,x):
        x= F.relu(self.f_connected1)
        x = F.relu(self.f_connected2)
        x =self.out(x)
        return x

In [34]:
## instantiate my ann model

torch.manual_seed(20) #every time you run the model this make sure that any random numbers like weights are same every time you run the code
model = ANN_Model()

In [ ]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=2, bias=True)
)>